In [1]:
## Demo Notebook example
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import pandas as pd
%pprint

Pretty printing has been turned OFF


In [2]:
### this is yff morning durations where two vehicles will be utilized , no pick up schedule provided 
### the problem is to utilize the two vehicles with the minimum duration possible 
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
dist=pd.read_excel('input/durations.xlsx')
window=pd.read_excel('input/time_windows.xlsx')

window

,serial,a,b
0,0,0,0
1,1,10,265
2,2,20,265
3,3,0,265
4,4,0,265
5,5,0,265
6,6,0,265
7,7,80,265
8,8,0,265
9,9,0,265


In [3]:
ab=[]
for i in range(0,dist.shape[0]):
   a=dist.loc[i,:].values
   a=a.tolist()
   ab.append(a)
ab

[[0, 15, 14, 19, 13, 10, 13, 12, 8, 10, 13, 31, 12, 13, 12, 13, 4, 8, 23, 11, 12, 11, 15, 10, 11, 10, 6, 14, 8, 0], [16, 0, 25, 6, 11, 19, 11, 20, 20, 14, 24, 43, 23, 15, 7, 11, 16, 12, 24, 22, 21, 9, 27, 22, 23, 22, 18, 21, 15, 15], [12, 26, 0, 30, 24, 20, 24, 22, 10, 20, 11, 30, 13, 24, 23, 24, 15, 18, 34, 9, 23, 22, 14, 9, 13, 6, 10, 24, 18, 12], [20, 6, 30, 0, 15, 23, 15, 25, 24, 18, 28, 47, 28, 20, 12, 15, 20, 16, 22, 27, 25, 14, 31, 26, 27, 26, 22, 24, 19, 20], [13, 11, 23, 15, 0, 14, 0, 16, 18, 11, 22, 41, 21, 11, 8, 0, 14, 8, 23, 20, 17, 10, 25, 19, 21, 20, 16, 17, 12, 13], [11, 19, 21, 23, 14, 0, 14, 2, 16, 14, 20, 39, 19, 7, 16, 14, 10, 11, 15, 18, 2, 15, 23, 17, 19, 18, 14, 6, 6, 11], [13, 11, 23, 15, 0, 14, 0, 16, 18, 11, 22, 41, 21, 11, 8, 0, 14, 8, 23, 20, 17, 10, 25, 19, 21, 20, 16, 17, 12, 13], [13, 21, 23, 25, 16, 2, 16, 0, 18, 15, 22, 40, 21, 8, 18, 16, 11, 12, 16, 20, 1, 17, 25, 19, 21, 20, 16, 8, 8, 13], [7, 21, 10, 25, 19, 16, 19, 18, 0, 15, 5, 28, 6, 19, 18, 19, 1

In [4]:
data={}
data['time_matrix']=ab
data['num_vehicles'] = 2
data['depot'] = 0
data['time_windows']=[]
for i in range(0,window.shape[0]):
    data['time_windows'].append((window['a'][i],window['b'][i]))


In [5]:
len(data['time_matrix'])


30

In [6]:
len(data['time_windows'])

30

In [7]:
data['time_windows']


[(0, 0), (10, 265), (20, 265), (0, 265), (0, 265), (0, 265), (0, 265), (80, 265), (0, 265), (0, 265), (140, 265), (0, 265), (0, 265), (0, 265), (30, 265), (40, 265), (50, 265), (60, 265), (70, 265), (80, 265), (0, 265), (0, 265), (120, 265), (0, 265), (0, 265), (90, 265), (0, 265), (120, 265), (0, 265), (0, 265)]

In [8]:
manager= pywrapcp.RoutingIndexManager(len(data['time_matrix']),data['num_vehicles'],data['depot'])

routing= pywrapcp.RoutingModel(manager)


In [9]:
def time_callback(from_index,to_index):
    from_node= manager.IndexToNode(from_index)
    to_node= manager.IndexToNode(to_index)
    return data['time_matrix'][from_node][to_node]

transit_callback_index= routing.RegisterTransitCallback(time_callback)

routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

time='Time'
routing.AddDimension( transit_callback_index
    , 260
    , 260, False, time)




True

In [10]:
time_dimension= routing.GetDimensionOrDie(time)
time_dimension.SetGlobalSpanCostCoefficient(1000)

data['time_windows'][0][0]

0

In [11]:
###Adding time windows consttraint

for node, time_window in enumerate(data['time_windows']):
    if node ==0:
        continue
    index= manager.NodeToIndex(node)
    time_dimension.CumulVar(index).SetRange(int(time_window[0]),int(time_window[1]))




In [12]:
for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(int(data['time_windows'][0][0]),
                                                int(data['time_windows'][0][1]))

for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))


In [13]:
    # Setting first solution heuristic.

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.seconds=300


In [14]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)



In [15]:
def get_routes(solution,routing,manager):
    routes=[]
    for node in range(routing.vehicles()):
        index= routing.Start(node)
        route= [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index= solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return(routes)
get_routes(solution,routing,manager)            
        
        
  

[[0, 29, 26, 24, 12, 23, 8, 11, 2, 25, 22, 19, 10, 0], [0, 21, 3, 1, 14, 6, 4, 15, 17, 9, 5, 7, 20, 18, 27, 13, 28, 16, 0]]

In [16]:
def get_time_windows(solution,routing,manager,dimension):
    all_list=[]
    for vehicle in range(routing.vehicles()):
        route_data=[]
        index= routing.Start(vehicle)
        route= manager.IndexToNode(index)
        dim= dimension.CumulVar(index)
        route_data.append([route,solution.Min(dim),solution.Max(dim)])
        while not routing.IsEnd(index):
            index= solution.Value(routing.NextVar(index))
            route= manager.IndexToNode(index)
            dim= dimension.CumulVar(index)
            route_data.append([route,solution.Min(dim),solution.Max(dim)])
        all_list.append(route_data)
    return all_list


get_time_windows(solution,routing,manager,time_dimension) 

[[[0, 0, 0], [29, 0, 0], [26, 6, 6], [24, 14, 14], [12, 23, 23], [23, 27, 27], [8, 51, 51], [11, 79, 79], [2, 109, 109], [25, 115, 115], [22, 126, 126], [19, 134, 134], [10, 140, 140], [0, 151, 151]], [[0, 0, 0], [21, 11, 11], [3, 25, 25], [1, 31, 31], [14, 38, 38], [6, 45, 45], [4, 45, 45], [15, 45, 45], [17, 60, 60], [9, 66, 66], [5, 79, 79], [7, 81, 81], [20, 82, 82], [18, 97, 97], [27, 120, 120], [13, 126, 126], [28, 134, 134], [16, 141, 141], [0, 146, 146]]]